In [0]:
# RNN也可以用在图片领域

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


mnist = input_data.read_data_sets('datasets', one_hot=True)

In [0]:
m_input = 28 # 输入1行，每行28个元素
max_line = 28 # 总共28行

lstm_size = 100 # 隐藏单元
m_classes = 10 # 10个分类
batch_size = 50
m_batch = mnist.train.num_examples//batch_size

In [0]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

weights = tf.Variable(tf.truncated_normal([lstm_size, m_classes], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[m_classes]))


def RNN(x, weights, biases):
    inputs = tf.reshape(x, [-1, max_line, m_input])
    # 基本的lstm cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    # final_state[0]为cell state，final_state[1]为hidden state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    #results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    results = tf.matmul(final_state[1], weights) + biases
    return results


prediction = RNN(x, weights, biases)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(m_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Epoch %d , Test Accuracy is %f' % (epoch, acc))

Epoch 0 , Test Accuracy is 0.834600
Epoch 1 , Test Accuracy is 0.898200
Epoch 2 , Test Accuracy is 0.919600
Epoch 3 , Test Accuracy is 0.934600
Epoch 4 , Test Accuracy is 0.929800
Epoch 5 , Test Accuracy is 0.943000
